# Selecting N0 : set of initial recipes to rank

- greedy algorithm : take 1st random recipe, then most dissimilar, etc.. stop until reached the desired number
- randomly take 1 recipe from 1 cluster

In [5]:
from pandas import Series, DataFrame
import pandas as pd
import numpy as np
import nltk
import re
import matplotlib.pyplot as plt
plt.style.use('ggplot')
import seaborn as sns
import numpy as np
from scipy.stats import norm
from sklearn.preprocessing import StandardScaler
from scipy import stats
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline

In [19]:
# Import useful files
recipes = pd.read_csv('database/recipe_final_utf8.csv')
correspondancetable = pd.read_csv('database/q.csv', index_col=0) # binary table with 0/1 if ingredient in recipe
correspondancetable.reset_index(drop=True, inplace=True)
recipes_type_cuisine = pd.read_csv('recipes-classification/recipes_by_type_cuisine.csv')
recipes_type_cuisine.set_index('recipe_id', inplace=True)
recipes_info_and_jaccoef = pd.read_csv('recipes_info_and_jaccoef.csv')

In [7]:
recipes_ing = recipes.merge(correspondancetable, left_index=True, right_index=True)
recipes_ing.set_index('recipe_id', inplace=True)

In [8]:
# Similarities arrays
from numpy import genfromtxt
array_sim_ing = genfromtxt('jaccard_recipes.csv', delimiter=',')
array_sim_type_cuisine = genfromtxt('sim_type_cuisine.csv', delimiter=',')

In [21]:
# Import the Jaccard matrix of the recipes dataset
df_jaccard = pd.read_csv('jaccard_recipes.csv',header=None)
df_jaccard.index += 1
df_jaccard.index.rename('recipe_id',inplace=True)
df_jaccard.columns = range(1,1525)

In [23]:
# Create new dataframe with Jaccard coefs + info about each recipe
recipes_jaccoef_info = recipes_ing.iloc[:,:5].merge(df_jaccard, how='inner', left_index=True, right_index=True)
recipes_jaccoef_info.head()

,id,title,Ingredients,Directions,numberofsteps,1,2,3,4,5,...,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524
recipe_id,,,,,,,,,,,,,,,,,,,,,
1,1584,Foul Moudammas,\r* 1 cup cooked fava beans or canned fava bea...,== Directions ==\r# Drain fava beans. Put in a...,5,1.000000,0.000000,0.100000,0.166667,0.090909,...,0.125000,0.333333,0.0,0.0,0.133333,0.055556,0.428571,0.000000,0.100000,0.071429
2,1624,Potato Dumplings,\r* 2 pounds [[potato]]es\r* [[all-purpose flo...,== Directions ==\r# Boil [[potato]]es until te...,11,0.000000,1.000000,0.100000,0.076923,0.090909,...,0.285714,0.000000,0.0,0.0,0.000000,0.055556,0.000000,0.083333,0.000000,0.071429
3,1636,Lebanese Basic Dough,\r* 5 cups plain [[flour]]\r* 1å_ cups tepid [...,== Directions ==\r# Sift the [[flour]] into a ...,10,0.100000,0.100000,1.000000,0.071429,0.625000,...,0.250000,0.181818,0.0,0.0,0.058824,0.111111,0.375000,0.076923,0.090909,0.142857
4,1655,Yakhnet Bathinjan,"\r* 16 oz black [[eggplant]]s, peeled, cubed\r...",== Directions ==\r# Sprinkle [[salt]] on [[egg...,11,0.166667,0.076923,0.071429,1.000000,0.066667,...,0.083333,0.230769,0.0,0.0,0.050000,0.045455,0.076923,0.062500,0.000000,0.187500
5,1660,Manakeesh bi Zaatar,\r* 2å_ cups plain [[flour]]\r* 1 tsp [[yeast]...,== Directions ==\r# Dilute the [[rice]] in a b...,5,0.090909,0.090909,0.625000,0.066667,1.000000,...,0.222222,0.166667,0.0,0.0,0.055556,0.105263,0.333333,0.071429,0.181818,0.133333


In [26]:
recipes_jaccoef_info.iloc[:,1+4]

recipe_id
1       1.000000
2       0.000000
3       0.100000
4       0.166667
5       0.090909
6       0.375000
7       0.272727
8       0.090909
9       0.000000
10      0.000000
11      0.000000
12      0.083333
13      0.333333
14      0.000000
15      0.000000
16      0.076923
17      0.625000
18      0.000000
19      0.100000
20      0.000000
21      0.125000
22      0.375000
23      0.250000
24      0.111111
25      0.333333
26      0.250000
27      0.083333
28      0.076923
29      0.000000
30      0.100000
          ...   
1495    0.181818
1496    0.100000
1497    0.090909
1498    0.100000
1499    0.000000
1500    0.250000
1501    0.000000
1502    0.066667
1503    0.111111
1504    0.200000
1505    0.000000
1506    0.200000
1507    0.000000
1508    0.125000
1509    0.142857
1510    0.125000
1511    0.300000
1512    0.000000
1513    0.111111
1514    0.000000
1515    0.125000
1516    0.333333
1517    0.000000
1518    0.000000
1519    0.133333
1520    0.055556
1521    0.428571
1522

In [6]:
print array_sim_ing, array_sim_ing.shape

[[ 1.          0.          0.1        ...,  0.          0.1         0.07142857]
 [ 0.          1.          0.1        ...,  0.08333333  0.          0.07142857]
 [ 0.1         0.1         1.         ...,  0.07692308  0.09090909
   0.14285714]
 ..., 
 [ 0.          0.08333333  0.07692308 ...,  1.          0.          0.        ]
 [ 0.1         0.          0.09090909 ...,  0.          1.          0.        ]
 [ 0.07142857  0.07142857  0.14285714 ...,  0.          0.          1.        ]] (1524, 1524)


## Selecting N0 : algorithm

1. Randomly select 1 recipe. Add the idx of this recipe to a list of selected indices.
2. Take most dissimilar recipe from this one. Add idx to list.
3. Repeat steps 1 and 2 with idx not in list.

In [11]:
A = array_sim_ing

In [16]:
A = array_sim_ing
N0 = 20
list_idx = []
idx = np.random.randint(A.shape[0], size=1)[0]
list_idx.append(idx)
dict_idx

while len(list_idx) < N0:
    last_idx = list_idx[-1]
    farthest_idx = np.argmin(A[last_idx,:])
    
    if farthest_idx not in list_idx:
        list_idx.append(farthest_idx)
    else:
        sorted_idx = np.argsort(A[last_idx,:], kind='quicksort', order=None)
        new_farthest_idx = next(i for i in sorted_idx if i not in list_idx)
        list_idx.append(new_farthest_idx)
        


print list_idx, len(list_idx)

[759, 5, 1, 0, 1098, 986, 8, 1003, 967, 453, 3, 1331, 867, 816, 1274, 991, 1147, 1407, 7, 609] 20


In [12]:
idx = np.random.randint(A.shape[0], size=1)[0]

In [13]:
farthest_idx = np.argmin(A[idx,:])
sorted_idx = np.argsort(A[idx,:], kind='quicksort', order=None)
print idx, farthest_idx, sorted_idx

453 3 [ 761  418  419 ..., 1317  949  453]


### Retrieve list of recipes N0 and their corresponding similarity

In [51]:
list_idx

[759,
 5,
 1,
 0,
 1098,
 986,
 8,
 1003,
 967,
 453,
 3,
 1331,
 867,
 816,
 1274,
 991,
 1147,
 1407,
 7,
 609]

In [62]:
list_recipes_idx = list_idx
df_N0 = recipes_jaccoef_info.iloc[list_recipes_idx,:].merge(recipes_type_cuisine, how='inner', left_index=True, right_index=True)
df_N0.groupby('predictions').count()

,id,title_x,Ingredients,Directions,numberofsteps,1,2,3,4,5,...,1517,1518,1519,1520,1521,1522,1523,1524,title_y,ingredients_clean_dumbwords
predictions,,,,,,,,,,,,,,,,,,,,,
french,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
greek,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
indian,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
italian,6,6,6,6,6,6,6,6,6,6,...,6,6,6,6,6,6,6,6,6,6
mexican,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
russian,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
southern_us,8,8,8,8,8,8,8,8,8,8,...,8,8,8,8,8,8,8,8,8,8


In [57]:
df_N0

,id,title_x,Ingredients,Directions,numberofsteps,1,2,3,4,5,...,1518,1519,1520,1521,1522,1523,1524,title_y,ingredients_clean_dumbwords,predictions
recipe_id,,,,,,,,,,,,,,,,,,,,,
1,1584,Foul Moudammas,\r* 1 cup cooked fava beans or canned fava bea...,== Directions ==\r# Drain fava beans. Put in a...,5,1.000000,0.000000,0.100000,0.166667,0.090909,...,0.000000,0.133333,0.055556,0.428571,0.000000,0.100000,0.071429,Foul Moudammas,cooked fava beans or canned fava beans lemon j...,italian
2,1624,Potato Dumplings,\r* 2 pounds [[potato]]es\r* [[all-purpose flo...,== Directions ==\r# Boil [[potato]]es until te...,11,0.000000,1.000000,0.100000,0.076923,0.090909,...,0.000000,0.000000,0.055556,0.000000,0.083333,0.000000,0.071429,Potato Dumplings,potatoes allpurpose flour as required nutmeg e...,southern_us
4,1655,Yakhnet Bathinjan,"\r* 16 oz black [[eggplant]]s, peeled, cubed\r...",== Directions ==\r# Sprinkle [[salt]] on [[egg...,11,0.166667,0.076923,0.071429,1.000000,0.066667,...,0.000000,0.050000,0.045455,0.076923,0.062500,0.000000,0.187500,Yakhnet Bathinjan,black eggplants peeled cubed lamb stew meat cu...,southern_us
6,1724,Labna,\r* 2 kg Greek [[yogurt]]\r* 75 ml extra virgi...,== Directions ==\r# Mix all ingredients togeth...,6,0.375000,0.000000,0.090909,0.153846,0.181818,...,0.000000,0.058824,0.000000,0.222222,0.000000,0.090909,0.066667,Labna,greek yogurt extra virgin olive oil salt lemon...,greek
8,1834,Kremithosoupa,\r* 2 cups [[onion]]s\r* å_ cup [[olive oil]]\...,"== Directions ==\r# Peel, wash and cube the [[...",4,0.090909,0.090909,0.300000,0.230769,0.272727,...,0.000000,0.117647,0.050000,0.333333,0.000000,0.083333,0.214286,Kremithosoupa,onions olive oil potatoes tomato sauce water m...,italian
9,1910,Toshiko's Broccoli Salad,"\r* 1 kg [[broccoli]], cut into florets\r* 2 t...",== Directions ==\r# Bring a large pot of [[wat...,6,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,Toshiko's Broccoli Salad,broccoli cut into florets tablespoons soy sauc...,indian
454,31899,South of the Border Chicken Soup,\r\r* 3 tablespoons [[vegetable oil]]\r* 3 (6&...,== Directions ==\r# Heat [[oil]] in a large st...,7,0.083333,0.083333,0.166667,0.000000,0.071429,...,0.000000,0.111111,0.157895,0.083333,0.000000,0.166667,0.058824,South of the Border Chicken Soup,tablespoons vegetable oil ampnbspinch tortilla...,mexican
610,46615,Jordan's Sticky Buns,\r* Rich's frozen dough rolls (ball size)\r* [...,== Directions ==\r# Lightly [[butter]] Bundt p...,8,0.000000,0.111111,0.000000,0.076923,0.000000,...,0.250000,0.000000,0.000000,0.000000,0.300000,0.000000,0.071429,Jordan's Sticky Buns,richs frozen dough rolls ball size raisins eno...,southern_us
760,48973,Stuffed Bass,\r* 1 whole 1.3&amp;nbsp;kg/3&amp;nbsp;lb stri...,"== Directions ==\r# Preheat the oven to 200C, ...",6,0.181818,0.083333,0.076923,0.133333,0.071429,...,0.083333,0.250000,0.100000,0.181818,0.066667,0.272727,0.058824,Stuffed Bass,whole .ampnbspkgampnbsplb striped bass cleaned...,italian


In [42]:
def get_similarity_with_previous(df):
    list_idx = list(df.index.values)
    for idx, row in df.iterrows():
        if idx ==0:
            sim = 0

        else:
            previous_idx = list_idx.index(idx)
            sim = df.iloc[idx,previous_idx+4] # only valid for dataframe recipes_jaccoef_info !!
        row['sim_with_previous'] = sim 
    return df       

In [43]:
get_similarity_with_previous(recipes_jaccoef_info.iloc[list_recipes_idx,:])

IndexError: single positional indexer is out-of-bounds

In [35]:
list_recipes_idx

[759,
 5,
 1,
 0,
 1098,
 986,
 8,
 1003,
 967,
 453,
 3,
 1331,
 867,
 816,
 1274,
 991,
 1147,
 1407,
 7,
 609]

In [49]:

tmp = recipes_jaccoef_info.iloc[list_recipes_idx,:]
tmp.iloc[:, :12]

,id,title,Ingredients,Directions,numberofsteps,1,2,3,4,5,6,7
recipe_id,,,,,,,,,,,,
760,48973,Stuffed Bass,\r* 1 whole 1.3&amp;nbsp;kg/3&amp;nbsp;lb stri...,"== Directions ==\r# Preheat the oven to 200C, ...",6,0.181818,0.083333,0.076923,0.133333,0.071429,0.000000,0.214286
6,1724,Labna,\r* 2 kg Greek [[yogurt]]\r* 75 ml extra virgi...,== Directions ==\r# Mix all ingredients togeth...,6,0.375000,0.000000,0.090909,0.153846,0.181818,1.000000,0.071429
2,1624,Potato Dumplings,\r* 2 pounds [[potato]]es\r* [[all-purpose flo...,== Directions ==\r# Boil [[potato]]es until te...,11,0.000000,1.000000,0.100000,0.076923,0.090909,0.000000,0.076923
1,1584,Foul Moudammas,\r* 1 cup cooked fava beans or canned fava bea...,== Directions ==\r# Drain fava beans. Put in a...,5,1.000000,0.000000,0.100000,0.166667,0.090909,0.375000,0.272727
1099,101659,Super Crunch Chicken,"\r\r* 8 boneless, skinless [[Chicken]] thighs ...",== Directions ==\r\r# Preheat oven to 375 degr...,11,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
987,53503,Boulanger Surprise,\r\r* 3 large boulangers\r* å_ lb large prawns...,== Directions ==\r\r# Remove the shells and ba...,10,0.214286,0.133333,0.058824,0.235294,0.055556,0.125000,0.312500
9,1910,Toshiko's Broccoli Salad,"\r* 1 kg [[broccoli]], cut into florets\r* 2 t...",== Directions ==\r# Bring a large pot of [[wat...,6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1004,53939,Haam am HÌ©e,"\r* 1 shoulder of [[ham]], lightly smoked, of ...",== Directions ==\r# Take a very large metal co...,9,0.000000,0.000000,0.142857,0.000000,0.125000,0.000000,0.000000
968,53283,Portuguese Bread Pudding,"\r\r* 8-slices of bread, cut into pieces\r* 2-...",== Directions ==\r\r# Mix all ingredients toge...,4,0.000000,0.111111,0.100000,0.000000,0.090909,0.000000,0.076923


In [37]:
list(tmp.index.values)

[760,
 6,
 2,
 1,
 1099,
 987,
 9,
 1004,
 968,
 454,
 4,
 1332,
 868,
 817,
 1275,
 992,
 1148,
 1408,
 8,
 610]

In [29]:
list_recipes_idx = list_idx
recipes_jaccoef_info.iloc[list_recipes_idx,:]

,id,title,Ingredients,Directions,numberofsteps,1,2,3,4,5,...,1515,1516,1517,1518,1519,1520,1521,1522,1523,1524
recipe_id,,,,,,,,,,,,,,,,,,,,,
760,48973,Stuffed Bass,\r* 1 whole 1.3&amp;nbsp;kg/3&amp;nbsp;lb stri...,"== Directions ==\r# Preheat the oven to 200C, ...",6,0.181818,0.083333,0.076923,0.133333,0.071429,...,0.090909,0.153846,0.000000,0.083333,0.250000,0.100000,0.181818,0.066667,0.272727,0.058824
6,1724,Labna,\r* 2 kg Greek [[yogurt]]\r* 75 ml extra virgi...,== Directions ==\r# Mix all ingredients togeth...,6,0.375000,0.000000,0.090909,0.153846,0.181818,...,0.111111,0.181818,0.000000,0.000000,0.058824,0.000000,0.222222,0.000000,0.090909,0.066667
2,1624,Potato Dumplings,\r* 2 pounds [[potato]]es\r* [[all-purpose flo...,== Directions ==\r# Boil [[potato]]es until te...,11,0.000000,1.000000,0.100000,0.076923,0.090909,...,0.285714,0.000000,0.000000,0.000000,0.000000,0.055556,0.000000,0.083333,0.000000,0.071429
1,1584,Foul Moudammas,\r* 1 cup cooked fava beans or canned fava bea...,== Directions ==\r# Drain fava beans. Put in a...,5,1.000000,0.000000,0.100000,0.166667,0.090909,...,0.125000,0.333333,0.000000,0.000000,0.133333,0.055556,0.428571,0.000000,0.100000,0.071429
1099,101659,Super Crunch Chicken,"\r\r* 8 boneless, skinless [[Chicken]] thighs ...",== Directions ==\r\r# Preheat oven to 375 degr...,11,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.142857,0.000000
987,53503,Boulanger Surprise,\r\r* 3 large boulangers\r* å_ lb large prawns...,== Directions ==\r\r# Remove the shells and ba...,10,0.214286,0.133333,0.058824,0.235294,0.055556,...,0.142857,0.187500,0.071429,0.000000,0.142857,0.000000,0.133333,0.111111,0.000000,0.222222
9,1910,Toshiko's Broccoli Salad,"\r* 1 kg [[broccoli]], cut into florets\r* 2 t...",== Directions ==\r# Bring a large pot of [[wat...,6,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1004,53939,Haam am HÌ©e,"\r* 1 shoulder of [[ham]], lightly smoked, of ...",== Directions ==\r# Take a very large metal co...,9,0.000000,0.000000,0.142857,0.000000,0.125000,...,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.000000,0.000000
968,53283,Portuguese Bread Pudding,"\r\r* 8-slices of bread, cut into pieces\r* 2-...",== Directions ==\r\r# Mix all ingredients toge...,4,0.000000,0.111111,0.100000,0.000000,0.090909,...,0.285714,0.000000,0.000000,0.250000,0.000000,0.000000,0.000000,0.083333,0.000000,0.071429
